# Minionology: HVSR with seismic nodes
### Skience2023 practical on HVSR, node installation, applications, Geopsy, continuous data analysis

##### Minion Gru's:
* Koen Van Noten ([@KoenVanNoten](https://github.com/KoenVanNoten))
* Thomas Lecocq ([@seismotom](https://github.com/ThomasLecocq))
* Raphael De Plaen

## Exercise 3:  
## The nature of the Bayrischzell valley infill
In this exercise, we will do a similar analysis as in Brussels (ex1), but now with data gathered along the Leitzach river valley, W of Bayrischzell. These seismic dataset was gathered between Sunday 27/02/2023 and Monday 28/03/2023 by Koen, Tom and Raph who installed 12 SmartSolo nodes across the valley. See red dashed line in the figure below.

##### Purpose
Analyse the ambient noise recordings either using Geopsy manually and making a cross-profile (see ex 1), or using an automatic analysis (ex. 2) to investigate the valley infill.

##### Data interpretation
To make an interpretation of the data, it is useful to have a look to the local geological map of Bayrischzel, which can be consulted here: 
https://www.bestellen.bayern.de/application/pictureSrv?SID=3274027&ACTIONxSESSxSHOWPIC(BILDxKEY:%2712052%27,BILDxCLASS:%27Artikel%27,BILDxTYPE:%27BildGross%27)=Z

and here:
https://geoportal.bayern.de/bayernatlas/?lang=de&topic=umwe&bgLayer=atkis&catalogNodes=110&layers=8885cab8-d186-4bfd-b61e-d419457649e8&E=723374.18&N=5285515.14&zoom=10&layers_opacity=0.6

__Infill Geology__:
* Postglacial Schoffer (Sand und Schoffer)
* Local morene mitt Wallform

__Bedrock Geology__:
* Hauptdolomite (HD)

<img src="Figures/Bayrischzell valley.png" width=800></img>

The seismic dataset gathered Bayrischzell can be downloaded here:
__TO DO__

Before playing with the HVSR data, let's have a look on data gathered.

### 0. Prequel: elevation data
First, read the topography cross-profile _'Barischzell_crossprofile.csv'_ in folder __in HVSR_ex3_Bay\__  

Plot it along distance using the cumulative distance with respect to 1st point. The first point is the most northern point in the valley.

In [ ]:
import pandas as pd
import numpy as np
import obspy
import os
import glob
import datetime
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.ticker as mticker
from obspy import read
from obspy.imaging.scripts.scan import Scanner
from obspy.geodetics.base import gps2dist_azimuth 
from scipy.interpolate import interp1d

# run the previous notebook
%run 00_HVSR_Minionology_definitions.ipynb

# use ipython notebook in a wider screen
from IPython.display import display, HTML # Widen the view
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
profile = r'HVSR_ex3_Bay\Barischzell_crossprofile.csv'
df_profile = pd.read_csv(profile)
lat_profile = df_profile['Lat']
lon_profile = df_profile['Lon']
Z_profile = df_profile["Z"]

# compute inter_distances
inter_distances = []
for nr in np.arange(0,len(df_profile)-1,1):
    inter_distances.append(gps2dist_azimuth(lat_profile[nr], lon_profile[nr], lat_profile[nr+1],lon_profile[nr+1])[0])
inter_distances = pd.Series(inter_distances)

# open a series, but start with distance 0
cumul = pd.Series([0])
# add the cumulative distances to the cumul list
d_cumul_profile = pd.concat([cumul,inter_distances.cumsum()], ignore_index = True)
#d_cumul

#inter_distances
fig, ax= plt.subplots(figsize=(15,5))
plt.plot(d_cumul_profile,Z_profile)
plt.ylabel('Altitude [m]', fontsize = 14)
plt.xlabel('Distance [m]', fontsize = 14)
plt.title('Bayrischzell HV profile', fontsize = 14)

### Let's start
Activate the necessary modules/def's first.

Scan the data 

In [ ]:
scanner = Scanner()
scanner.parse("HVSR_ex3_Bay")  
scanner.plot() 

### 1. Manual geopsy HVSR dataprocessing
In the first solution, solve the HV data in Geopsy manually and save the .hv file

Use following parameters (or play with them):
* __Length__: 120s
* __Overlap__: 50%
* __Relative treshhold__: 70%
* __Taper__: 5% Tukey
* __Konno-Omachi smoothing__: 40%
* __Squared Average__
* __Output__: 0.20Hz - 50 Hz
* __Step Count__: 500

After computation, manually clean the H/V curve by _Reject time windows_ (right click on the graph) and select those curves that deviate from the mean curve. Then recompute the H/V curve by pressing the black arrow next to _select_ -> _clear_ -> _Remove grayed_ and press _Start_ again.

To save the .HV results do a _Tools_ -> _Save Results_ and save it in the  __HVSR_ex3_Bay\Analysed__ folder

### 2. Reading theBayrischzell HVSR database file
All the node metadata information has been preloaded in an HVSR database file. The HVSR database file is available in the __HVSR_ex3_Bay__ folder.

In [ ]:
HV_database = r'HVSR_ex3_Bay\HVSR_database_Bayrischzell.csv'
HV_db_folder, HV_db_name = os.path.split(HV_database)[0], os.path.split(HV_database)[1]
print(HV_db_folder, HV_db_name)

Load the HVSR database into a pandas database. 

In [ ]:
db_HVSR = pd.read_csv(HV_database, encoding='latin')
db_HVSR.head()

Compute the distance of each node to the first node using the Obspy _obspy.geodetics.base.gps2dist_azimuth_ function and load it to an inter_distance array

inter_distance = [] 

In [ ]:
from obspy.geodetics.base import gps2dist_azimuth 

lat = db_HVSR['Lat']
lon = db_HVSR['Lon']

inter_distances = []
for nr in np.arange(0,len(db_HVSR)-1,1):
    inter_distances.append(gps2dist_azimuth(lat[nr], lon[nr], lat[nr+1],lon[nr+1])[0])
inter_distances = pd.Series(inter_distances)
inter_distances

Make a cumulative distance series (d_cumul) with _pd.series.cumsum()_

d_cumul value of the first node is distance to the first node of the Bayrischzell profile. Compute this distance and add that  as first distance to d_cumul with pd.concat 

In [ ]:
#distance of first node to first point of the profile
d_first_node = gps2dist_azimuth(lat_profile[0], lon_profile[0], lat[0],lon[0])[0]

# open a series, and add the first node distance to it 
cumul = pd.Series(d_first_node)

# add the cumulative distances + d_first_node to the cumul list
d_cumul = pd.concat([cumul,inter_distances.cumsum()+d_first_node], ignore_index = True)
d_cumul

Add the cumulative distance column to the pandas dataframe

In [ ]:
db_HVSR['d_cumul'] = d_cumul
db_HVSR.head()

### 3. Filling the HVSR database file with .hv data

By using Geopsy, the HVSR data has been saved as .hv files in the _HVSR_ex1_Bru\Analysed_ folder. We will extract and add it to the HVSR database. The script in below (Van Noten _et al._ 2022) reads the HVSR database and extracts all the necessary data in the .hv files being:

* __f0 min__: f0_win/stddev (from GEOPSY)
* __f0_win__: average resonance frequency by taking the f0 of each individual window and averaging all f0 values from these windows (from GEOPSY)
* __f0 average__: scanning the average curve and identifying the frequency at which the maximum amplitude occurs (from GEOPSY)
* __f0_ip__: resonance frequency computed after interpolating the HV-Amplitude graph using python. This is useful if you forgot to adapt the Step Count
* __f0_ip_diff__: difference between f0_ip and f0_win
* __error__: standard deviation on f0 (from GEOPSY)
* __f0 max__: f0_win.stddev (from GEOPSY)
* __A0__: maximum amplitude (from GEOPSY)
* __nw__: number of windows (from GEOPSY)

Reference:
https://github.com/KoenVanNoten/HVSR_to_virtual_borehole/blob/master/Get%20f0s%20from%20geopsy%20hv%20files.py

In [ ]:
db_HVSR.head()

In [ ]:
# read the database file in which all the names of the .hv measurements are stored
HV_folder = os.path.join(HV_db_folder, 'Analysed')  #folder containing all .hv data

#### Initializing empty columns that need to be filled from the Geopsy .hv files
for _ in ["f0_min", "f0_win", "f0_avg", "f0_int", "f0_int_diff", "error", "f0_max", "A0", "nw"]:
    db_HVSR[_] = 0.

# specify a name for the HVSR + HV params database
out_file = os.path.splitext(HV_db_name)[0] + "_f0_from_hv.csv"

#### loop through each .hv datafile
for id, row in db_HVSR.iterrows():
    HV_file = os.path.join(HV_folder, row["ID"] + ".hv")
    print(HV_file)
    
    # get all params from the HV file
    f0_avg, f0_win, error, A0, nw_avg, nw_win, f_min, f_max = get_params_from_HV(HV_file)
    
    # get interpolated f0 and A0 from the HV file
    f0_int, A0_int, f0_int_diff = get_interpolated_values_from_HV(HV_file, 15000, f0_win)

    #write all data to the database file
    #write_HVline_to_db?
    write_HVline_to_db(db_HVSR, f_min, f0_win, f0_avg, f0_int, f0_int_diff, error, f_max, A0, nw_win)


db_HVSR.to_csv(os.path.join(HV_db_folder, out_file), index = False)

We can now check how the map looks like and color it by whatever parameter you want. e.g. f0

In [ ]:
# create the HV plot
fig, ax= plt.subplots()
#Checking how the profile looks like - Use the Belgian Lambert72 projection column 
scatter = plt.scatter(db_HVSR['UTM-X'], db_HVSR['UTM-Y'], c=db_HVSR['f0_avg'],  cmap = 'viridis')
cb = plt.colorbar(scatter, orientation='vertical')
cb.set_label('Resonance frequency', backgroundcolor = 'white')
plt.scatter(db_HVSR['UTM-X'][0], db_HVSR['UTM-Y'][0], c='red', label='first node: %s'%(db_HVSR["ID"][0]))
ax.axis('equal')
plt.xticks(rotation=90)
plt.legend()

### 4. Creating an HVSR cross-profile
We want to create an HVSR profile showing the variation of resonance frequency along the profile where the nodes where installed. We will create this distance plot using the cumulative distances computed earlier. Herefore we need to link each .hv file with its cumulative distance point.

In [ ]:
# create plot
fig, ax= plt.subplots(figsize=(16,6))

# We will loop over the HVSR database using pandas groupby to plot each HV profile along the distance. 
for id, group in db_HVSR.groupby("H/V"):
    
    # loop over the .hv files
    for id,line in group.iterrows():
        ID = line.ID + ".hv"
        HV_file = os.path.join(HV_db_folder, 'Analysed', ID)
        
        ### load necessary data
        Freq, A0, A_min, A_max = read_HV(HV_file)
        plt.plot(A0, Freq, c='grey', alpha = 0.5)

        # scatterplot a dot on f0 and A0      
        f0_curve, A0_curve, A0_min_curve, A0_max_curve = get_params_from_HV_curve(HV_file)
        plt.scatter(A0_curve, f0_curve, c = 'red', edgecolors = 'red', alpha = 0.5, zorder = 10, 
            label='$f_0$: %s Hz \n$A_0$: %s'%(round(f0_curve,2), round(A0_max_curve,1)))
        
        # or scatterplot a dot on f0 and A0 from the HV file
        f0_avg, f0_win, error, A0, nw_avg, nw_win, f_min, f_max = get_params_from_HV(HV_file)
        plt.scatter(A0, f0_avg, c = 'red', edgecolors = 'green', alpha = 0.5, zorder = 10 ,
                    label='$f_0$: %s Hz \n$A_0$: %s'%(round(f0_curve,2), round(A0_max_curve,1)))

        # or get the values from a partial part of the HV curve and scatterplot
        f_range, A_range, A_min_range, A_max_range = get_params_from_partial_HV_curve(HV_file, 10 , 100)
        plt.scatter(A_range, f_range, c = 'blue', edgecolors = 'grey', alpha = 0.8, zorder = 10, 
                    label='$f_0$: %s Hz \n$A_0$: %s'%(round(f_range,2), round(A_range,1)))
ax.set_yscale('log')
plt.xlabel('H/V amplitude', fontsize = 14)
plt.ylabel('Frequency ([Hz])', fontsize = 14)

In [ ]:
#Let's read one
HV_file = r'HVSR_ex3_Bay\Analysed\BE_01974.hv'

# create the plot
fig, ax= plt.subplots(figsize=(10,5))

# get the HV values and plot
Freq, A, A_min, A_max = read_HV(HV_file)
plot_HV(Freq, A, A_min, A_max, f0_curve, A0_curve,  A0_min_curve, A0_max_curve, 'manual', 'blue')

# or get the values from a partial part of the HV curve and scatterplot
f_range, A_range, A_min_range, A_max_range = get_params_from_partial_HV_curve(HV_file, 1 , 3)
plt.scatter(A_range, f_range, c = 'blue', edgecolors = 'grey', alpha = 0.8, zorder = 10, 
                    label='$f_0$: %s Hz \n$A_0$: %s'%(round(f_range,2), round(A_range,1)))

plt.title(os.path.splitext(os.path.split(HV_file)[1])[0], fontsize = 14)
plt.grid(ls='--')
ax.set_yscale('log')
plt.xlabel('H/V amplitude', fontsize = 14)
plt.ylabel('Frequency ([Hz])', fontsize = 14)
plt.legend()

In [ ]:
# We will loop over the HVSR database using pandas groupby to plot each HV profile along the distance. 
for id, group in db_HVSR.groupby("H/V"):
    
    # loop over the .hv files
    for id,line in group.iterrows():

        ID = line.ID + ".hv"
        HV_file = os.path.join(HV_db_folder, 'Analysed', ID)
        
       # create the plot
        fig, ax= plt.subplots(figsize=(10,5))

        # get the HV values and plot
        Freq, A, A_min, A_max = read_HV(HV_file)
        f0_curve, A0_curve, A0_min_curve, A0_max_curve = get_params_from_HV_curve(HV_file) # Get the HV params from the HV plot
        plot_HV(Freq, A, A_min, A_max, f0_curve, A0_curve,  A0_min_curve, A0_max_curve, 'manual', 'blue')
        
        plt.scatter(A0_curve, f0_curve, c = 'red', edgecolors = 'red', alpha = 0.5, zorder = 10, 
            label='$f_0$: %s Hz \n$A_0$: %s'%(round(f0_curve,2), round(A0_max_curve,1)))
        

        plt.title(os.path.splitext(os.path.split(HV_file)[1])[0], fontsize = 14)
        plt.grid(ls='--')
        ax.set_yscale('log')
        plt.xlabel('H/V amplitude', fontsize = 14)
        plt.ylabel('Frequency ([Hz])', fontsize = 14)
        plt.legend()

### Create the HV cross-profile
Now, let's make the HVSR profile with distance.

__Strategy__: this profile is made by
* finding the largest amplitude of all HV curves
* normalising all amplitudes to this largest amplitude
* converting the amplitude to distance (hack-solution) so it can be plotted 

__Possibilities__:
We can either make a 
* frequency-distance plot to show the HV curve variation
or convert the HV curve to depth by using:
* any information someone has on the velocity profile: 6C ping __Felix__ and __Sabrina__?
* or using a fixed wild guess velocity
* or using a powerlaw equation between f0 and depth in the form of h = _a_ * np.power(f0,_b_), with _a_ and _b_ fixed parameters set to a certain area (e.g. __Heiner__ checking all boreholes in the Bayrischzell valley and doing HVSR)


In [ ]:
HV_database = r'HVSR_ex3_Bay\HVSR_database_Bayrischzell_f0_from_hv.csv'

db_HVSR = pd.read_csv(HV_database, encoding='latin')

### Choose which profile you want: 
freq_profile = False ## frequency profile of all HVs

depth_profile_fixed = False #depth profile with frequency converted to depth with a fixed velocity 
Vs = 350 #m/s

depth_profile_powerlaw = True #depth profile with frequency converted to depth using a powerlaw equation
a_pw = 88.631     # a value of the powerlaw
b_pw = -1.683    # b value of the powerlaw

### Give the exaggeration value to exaggerate the horizontal scale of the HV plots
### e.g. h_exa = 1 will give virtual boreholes
h_exa = 80

#######################
## Main program
#######################
# create plot
fig, ax= plt.subplots(figsize=(16,6))

# Create empty arrays to get all the HV data
Freqs= []
A0s = []
Amins = []
Amaxs = []
f0s_db = []
A0s_db = []

max_amp = 0

 
# loop over the .hv files
for id, line in db_HVSR.iterrows():
        ID = line.ID + ".hv"
        HV_file = os.path.join(HV_db_folder, 'Analysed', ID)
        
        ### load necessary data
        Freq, A0, A_min, A_max = read_HV(HV_file)
        Freqs.append(Freq)
        A0s.append(A0)
        Amins.append(A_min)
        Amaxs.append(A_max)
        # get the maximum amplitude of all HV files to normalise the max amplitude later
        if A0.max() > max_amp:
            max_amp = A0.max()
        f0s_db.append(line.f0_avg)
        A0s_db.append(line.A0)
        
# Normalise the amplitudes and min and max (to potentially plot the error on f0)
A0s = pd.Series(A0s)/max_amp
Amins = pd.Series(Amins)/max_amp
Amaxs = pd.Series(Amaxs)/max_amp

# create a pandas db for plotting all curves
db_HVs = pd.DataFrame({'Freqs':Freqs, 'A0s':A0s, 'Amins':Amins, 'Amaxs':Amaxs, 
                       'd_cumul':db_HVSR["d_cumul"], 'Z':db_HVSR["Z"], 'ID':db_HVSR["ID"]})
for k, _ in db_HVs.iterrows():        
        # convert amplitudes to distances

        arrays = []
        for ar in [_.A0s, _.Amins, _.Amaxs]:
            ar *= h_exa
            ar += _.d_cumul
            arrays.append(ar)
        # if you want a frequency plot y = frequency values        
        if freq_profile:
            y = _.Freqs
            plt.text(_.d_cumul, 0.6, _.ID, rotation = 'vertical', c='gray', zorder = 10)
        
        # if you want a depth profile with a fixed Vs,  y = Vs / 4*f0            
        if depth_profile_fixed:
            y = Vs / (_.Freqs * 4)
            y= _.Z - y
            plt.text(_.d_cumul, 685, _.ID, rotation = 'vertical', c='gray', zorder = 10)
    
        # if you want a depth profile with a powerlaw conversion,  y = powerlaw            
        if depth_profile_powerlaw:
            y = a_pw * np.power(_.Freqs,b_pw)
            y= _.Z - y
        
        # color the lines
        colorline(arrays[0], y, _.A0s, cmap='viridis', linewidth=5)
        colorbar = colorline(arrays[0], y, _.A0s, cmap='viridis', linewidth=5)        
        
        # scatterplot a dot on f0 and A0
        Amax_f0 = np.max(arrays[0])
        Fmax_f0 = y[np.argmax(arrays[0])]
        plt.scatter(Amax_f0, Fmax_f0, c = 'red', edgecolors = 'red', alpha = 0.5, zorder = 10)
        

plt.gca().invert_yaxis()


if freq_profile:
        plt.ylabel("Frequency [Hz]",  fontsize=14)
        ax.set_yscale('log')
        plt.ylim(0.5,100)
else:
        plt.ylabel("Depth [m]",  fontsize=14)
        plt.ylim(680,820)

plt.xlabel("Distance [m] & Normalized Amplitude",  fontsize=14)
id = '%s '%(db_HVSR["Comment"][0].split('_')[0])
plt.title('H/V spectral ratio analysis - %s'%id,  fontsize=16)

plt.xlim(0,1000)
plt.grid()
plt.plot(d_cumul_profile,Z_profile, linewidth=2, c='black')
#plt.savefig('Figures/HVSR_profile_Brussels.png', dpi=150)

### What about the variability of each of the H/V curve(s)?


In [ ]:
# Linux
geopsy_exe = "geopsy-hv"

# Windows - locate the Geopsypack.win64-3.4.2 folder
geopsy_exe = 'C:/Users/koenvn/geopsypack-win64-3.4.2/bin/geopsy-hv.exe'

# For Mac, use a Windows (sorry :-))

# test the help file
#!{''.join(geopsy_exe)} -help

In [ ]:
# overall processing lengths (time for which we compute the total HV-curve)
#process_len = 3600. # [s] standard 1 hour
process_len = 1800. # [s] 0.5 hour

# window length for each HV curve
time_window_len = 60. # [s] standard 60 or 120s

#overlapping windows
win_overlap = 50

# threshold and percentage
threshold = 'RelativeSampleThreshold'
threshold_pct = 0.7

#Smoothing (in digits)
KO = 0.4

# lower frequency bound
min_freq = 0.5 # [Hz]

# upper frequency bound
max_freq = 100 # [Hz]

# how to calculate horizontals (Squared, Energy, Azimuth, Geometric)
horizontals_method = 'Squared' # standard 'Squared'

##HORIZONTAL_AZIMUTH is used only when HORIZONTAL_COMPONENTS== 'Azimuth'
azimuth = 0

# in case you want run the HV rotate module (see exercise 4)
want_rotation = False # True, False
rotation_steps = 10 # [degree]

# Using the Cox et al. 2020 filtering 
#FREQUENCY_WINDOW_REJECTION_MINIMUM_FREQUENCY
rej_min_freq = 0.5
#FREQUENCY_WINDOW_REJECTION_MAXIMUM_FREQUENCY
rej_max_freq = 50
#FREQUENCY_WINDOW_REJECTION_STDDEV_FACTOR
rej_stdev = 1.8
#FREQUENCY_WINDOW_REJECTION_MAXIMUM_ITERATIONS
rej_it = 500

In [ ]:
# Give waveforms
wf = "HVSR_ex3_Bay\\Raw_Data\\453001013.1.2023.02.26.14.40.00.000.*.miniseed"

node_ID = os.path.split(wf)[-1].split('.')[0] # Better to have BE.00617 in the folder name or the full node ID name??
print(node_ID)

# Give outputfolder where to save the .hv files
output_folder = 'HVSR_ex3_Bay/Auto_Analysed/'

In [ ]:
## Get the station info
st = read(wf)
tr = st[0]
start = tr.stats.starttime
end = tr.stats.endtime
station = '%s_%s'%(tr.stats.network, tr.stats.station)
print('station: %s'%station)

delta = end-start
print("start: %s"%start)
print("end: %s"%end) 
print("We will get %s .hv files of %ss length out of the stream"%(int(delta/process_len), process_len))

for i in np.arange(start, end, process_len):
        time = i
        print(time)
        tStart = '%s%02d%02d%02d%02d%#05.2f'%(time.year, time.month, time.day, time.hour, time.minute, time.second)
        tStart_hv = '%s%02d%02d%02d%02d%#02d'%(time.year, time.month, time.day, time.hour, time.minute, time.second)
        endtime = time + process_len
        tEnd = '%s%02d%02d%02d%02d%#05.2f'%(endtime.year, endtime.month, endtime.day, endtime.hour, endtime.minute, endtime.second)
        
        # get the empty auto-paramString
        paramsString = get_paramString()
        
        # adapt an auto-PARAM file with the given parameters so that for each processing loop the same param is used
        with open('geopsy-hv-auto.params', 'w') as f:
            f.write(paramsString.format(
                # Select start and end time from waveform
                tStart=tStart,
                # Select end time from waveform
                tEnd=tEnd,
                # Adapt the parameters of previously chosen parameters 
                threshold=threshold,
                threshold_pct=threshold_pct,
                KO=KO,
                winLen=time_window_len,
                overlap=win_overlap,
                minFreq=min_freq,
                maxFreq=max_freq,
                horizontals=horizontals_method,
                azimuth=azimuth,
                rotSteps=rotation_steps,
                rej_min_freq=rej_min_freq, 
                rej_max_freq=rej_max_freq, 
                rej_stdev=rej_stdev,
                rej_it = rej_it
                ))
        
        ### Make a folder for each station output
        os.makedirs(output_folder+node_ID, exist_ok=True)
        out_folder = ''.join(output_folder+node_ID)
        
        ### Run geopsy for each step in the loop
        !{''.join(geopsy_exe)} -hv {''.join(wf)} -param geopsy-hv-auto.params -o {''.join(output_folder+node_ID)}
        
        ### Rename the .hv output file and the .log to save a unique files for each processed process_len
        ### saving the .log files is useful as these can be loaded in Geopsy to manually check the processed data 
        os.renames(os.path.join(output_folder, '{0}/{1}.hv'.format(node_ID, station)), os.path.join(output_folder, '{0}/{1}.{2}.hv'.format(node_ID, station, tStart_hv)))
        os.renames(os.path.join(output_folder, '{0}/{1}.log'.format(node_ID, station)), os.path.join(output_folder, '{0}/{1}.{2}.log'.format(node_ID, station, tStart_hv)))

        # if want_rotation is selected, also the HV rotate module will be executed and stored in the outfolder  
        if want_rotation:
            
            # run the geopsy rotation
            !{''.join(geopsy_exe)} -rotate {''.join(wf)} -param geopsy-hv-auto.params -o {''.join(output_folder+node_ID)}
            
            ### Rename the .hv.grid output file
            os.renames(os.path.join(output_folder, '{0}/{1}.hv'.format(node_ID, station)), os.path.join(output_folder, '{0}/{1}.{2}.grid'.format(node_ID, station, tStart_hv)))
            
        print('**********************************')

print('Job Done')

## Visualise Bayrischzell auto_results

### HV variability of the node with time

In [ ]:
# give the output folder containing all subfolders with continuous hv files
output_folder = 'HVSR_ex3_Bay\Auto_Analysed'

###################################################################""
# create the continuous HV plot
fig, ax1= plt.subplots(figsize=(10,5))

# loop over the subfolders
folders = glob.glob("%s/453001013/"%output_folder, recursive = True)
print(folders)

for i in folders:

    HV_files = glob.glob('%s/*.hv'%i)
    
    # loop over the HV files
    for i,j in zip(HV_files, np.arange(0,len(HV_files),1)):
        node_ID = os.path.split(i)[-1].split('.')[0]
        # read the time from the HV name
        time = os.path.split(i)[1].split('.')[1]
        year = time[0:4]
        d = datetime.datetime(int(time[0:4]),int(time[4:6]), int(time[6:8]), int(time[8:10]), int(time[10:12]), int(time[12:14]))
        print(i, d)
    
        Freq, A, A_min, A_max = read_HV(i)
        plt.plot(A+j, Freq, c='gray', alpha=0.6)
        
        f0_curve, A0_curve, A0_min_curve, A0_max_curve = get_params_from_HV_curve(i)
        plt.scatter(A0_curve+j, f0_curve, c = 'red', edgecolors = 'red', alpha = 0.5, zorder = 10, 
            label='$f_0$: %s Hz \n$A_0$: %s'%(round(f0_curve,2), round(A0_max_curve,1)))
        

    plt.ylim(2,50)
    ax1.set_yscale('log')
    ax1.set_xlabel('Some meaningless value (time)', fontsize = 14)
    ax1.set_ylabel('Frequency [Hz]', fontsize = 14)
    ax1.grid(ls='--', axis='y', which='both')
    plt.subplots_adjust(hspace=0.1)
    #ax1.tick_params(labelbottom=True)
    plt.title('HVSR variability through time', fontsize = 14)

    plt.xticks(rotation = 90)
    

### HV variability of all nodes

In [ ]:
# give the output folder containing all subfolders with continuous hv files
output_folder = 'HVSR_ex3_Bay\Auto_Analysed'

###################################################################""
# create the continuous HV plot
fig, ax1= plt.subplots(figsize=(10,5))

# loop over the subfolders
folders = glob.glob("%s/*/"%output_folder, recursive = True)
print(folders)

for i in folders:
    
    f0_wins = []
    f0_avgs = []
    A0s = []
    times = []
    errors_min, errors_max = [], []
    HV_files = glob.glob('%s/*.hv'%i)
    
    # loop over the HV files
    for i in HV_files:
        node_ID = os.path.split(i)[-1].split('.')[0]
        # read the time from the HV name
        time = os.path.split(i)[1].split('.')[1]
        year = time[0:4]
        d = datetime.datetime(int(time[0:4]),int(time[4:6]), int(time[6:8]), int(time[8:10]), int(time[10:12]), int(time[12:14]))
        print(i, d)

        f0_avg, f0_win, error, A0, nw_avg, nw_win, f_min, f_max = get_params_from_HV(i)

        f0_avgs.append(f0_avg)
        A0s.append(A0)
        f0_wins.append(f0_win)
        errors_min.append(f0_avg-f_min)
        errors_max.append(f_max-f0_avg)
        times.append(d)

    plt.scatter(times, f0_avgs, ls='-', label = node_ID)    
    plt.errorbar(times, f0_avgs, xerr=None, yerr = (errors_min, errors_max), c='grey',  alpha=0.5, zorder=-1, ls='none')   
    ax1.set_yscale('log')
    ax1.yaxis.set_minor_formatter(mticker.ScalarFormatter())
    ax1.yaxis.set_major_formatter(mticker.ScalarFormatter())
    ax1.set_xlabel('Datetime', fontsize = 14)
    ax1.set_ylabel('Frequency [Hz]', fontsize = 14)
    ax1.legend(bbox_to_anchor=(1.05, 0), loc="lower left", borderaxespad=0, ncol=1)
    ax1.grid(ls='--', axis='y', which='both')
    plt.subplots_adjust(hspace=0.1)
    ax1.tick_params(labelbottom=True)
    plt.title('HVSR variability', fontsize = 14)

    #Format the xaxis date
    from matplotlib.dates import DateFormatter, DayLocator, HourLocator

    ax1.xaxis.set_major_locator(HourLocator())
    ax1.xaxis.set_major_formatter(DateFormatter("%H:%M"))
    plt.xticks(rotation = 90)
plt.show()